### build-mysql-database
CONSTROI BASE DE DADOS NO MYSQL POR MEIO DE DOCUMENTOS DA COLEÇÃO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import pymysql
import getpass

#import paramiko

import psycopg2
from sshtunnel import SSHTunnelForwarder
import time

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

path = "C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/"
outputs = "C:/Users/marcelo.ribeiro/Documents/outputs/"

files = [f for f in os.listdir(path)]
fullpath_list = []
fullpath = ''
for file in files:
    fullpath = path+file
    fullpath_list.append(fullpath)

In [3]:
files[0:4]

['AAS_mre_ag_1973.11.20_doc_I-1.txt',
 'AAS_mre_ag_1973.11.20_doc_I-10.txt',
 'AAS_mre_ag_1973.11.20_doc_I-100.txt',
 'AAS_mre_ag_1973.11.20_doc_I-101.txt']

In [7]:
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           passwd=pass_mysql, 
                           db='CPDOC_AS')
    curs = conn.cursor()
    sql = "select * from marcelo_draft"
    curs.execute(sql)
    rows = curs.fetchall()
    print(rows)

········
········
()


In [ ]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Captura datas
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
for file in files[0:10]:
    date = re.sub('.*(19\d\d.*)_.*_.*', r'\1', file)
    print(date)

In [ ]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
insere id de documentos e corpo de texto na base mysql
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
cur.execute("CREATE DATABASE IF NOT EXISTS test;")
cur.execute("USE test;")
cur.execute('''CREATE TABLE IF NOT EXISTS test_table 
               (text VARCHAR(200), id TEXT, body TEXT, date TEXT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    
for txt in fullpath_list:
    txt_date = re.sub('.*(19\d\d.*)_.*_.*', r'\1', txt)
    txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)
    with open(os.path.join(path,txt), 'r', encoding='utf-8') as f:
        txt_body = f.read()
        INSERT INTO test VALUES (varchar?, txt_id, txt_body, txt_date)


In [ ]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Classifica o corpus de acordo com: tipo textual (para separar textos sujos, como letra cursiva, foto, fonte alternativa) e idioma
pt = 1
de = 2
en = 3
es = 4
fr = 5
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
doc_class = []
lang_class = []
count_doc = 0
percentil = int(len(onlyfiles)/100)
for txt in onlyfiles[0:]:
    #print(txt)
    with open(os.path.join(path,txt), 'r', encoding='latin-1') as f:
        if count_doc % percentil == 0: print(int(count_doc/percentil),'% done')
        if count_doc % (percentil-1) == 0: clear_output()
        texto = f.read()
        texto_pt = texto_en = texto_es = texto_fr = texto
        texto_split = re.split('\.|\[|\]|\(|\)|\?|\/|\n|\*|\:|\%|\d+|\ª|\º', texto)
        #print(texto_split)
        count = pt_count = en_count = es_count = fr_count = de_count = 0
        total_count = 0
        for phrase in texto_split:
            total_count += 1
            if not re.search('[^\W\d_]', phrase): continue
            language = detect(phrase)
            #print('PHRASE', phrase, 'LANGUAGE', language)
            total_count += 1
            if language == 'pt':
                pt_count += 1
            if language == 'en':
                en_count += 1
            if language == 'es':
                es_count += 1
            if language == 'fr':
                fr_count += 1
            if language == 'de':
                de_count += 1
        lang_count = pt_count + en_count + es_count + fr_count + de_count        
        #print(total_count, lang_count, 'proportion', lang_count/total_count)
        if lang_count/total_count < 0.03: 
            doc_class.append(0)
        else: doc_class.append(1)
        lang_class.append(0)
        if de_count/total_count > 0.1: 
            lang_class[-1] = 2
        if fr_count/total_count > 0.1: 
            lang_class[-1] = 5
        if es_count/total_count > 0.1: 
            lang_class[-1] = 4
        if en_count/total_count > 0.1: 
            lang_class[-1] = 3
        if pt_count/total_count > 0.1: 
            lang_class[-1] = 1
        else: 

        #print(pt_count/total_count)
        #print(en_count/total_count)
        #print(es_count/total_count)
        #print(fr_count/total_count)
        #print(de_count/total_count)
        
        #print('len',len(texto_split))
        #print(doc_class)